# 3. Pré-processamento

Nosso modelo só pode prever os jogos com base em dados passados dos times, não a partir das variáveis do jogo a ser previsto (até porque a previsão ocorre antes do jogo). O algoritmo deve se basear apenas nas informações passadas

In [152]:
import pandas as pd
import datetime as dt
import numpy as np

In [5]:
df = pd.read_csv('dados_brasileirao_tratados.csv') 
brasileirao = pd.read_csv('jogos_brasileirao.csv')

## Calcular média móvel

Para uma análise de resultado normalmente leva-se em conta os últimos 3 ou 5 jogos disputados pelas equipes. Por conta disso, calcularemos a **média móvel dos últimos 5 jogos de cada time**

### Juntando dataframes

Vou precisar atribuir o ano dos jogos às linhas na base de dados usada para fazer o treinamento. Isso porque terei de agrupar por ano os dados para calcular a média móvel

In [10]:
# Criando uma coluna 'ano' com o ano em que cada jogo aconteceu
try:
    brasileirao['ano'] = pd.to_datetime(brasileirao['data'].str.title(), dayfirst=True).dt.year
    
except AttributeError:
    print('A conversão já foi feita')

In [11]:
brasileirao.rename(columns={'ID': 'partida_id'}, inplace=True)

In [12]:
#Fazendo a junção dos dataframes
df_atualizado = pd.merge(brasileirao[['partida_id', 'ano']], df, on='partida_id')

In [13]:
df_atualizado.head()

,partida_id,ano,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
0,6126,2018,1,Cruzeiro,12.0,2.0,0.40,425.0,0.81,20.0,3,0,0,7
1,6126,2018,1,Gremio,6.0,2.0,0.60,649.0,0.86,15.0,1,1,3,4
2,6127,2018,1,Vitoria,21.0,6.0,0.66,545.0,0.88,24.0,3,0,2,4
3,6127,2018,1,Flamengo,10.0,5.0,0.34,286.0,0.72,10.0,2,1,2,5
4,6128,2018,1,Santos,22.0,3.0,0.54,532.0,0.90,7.0,0,0,1,4


### Calculando a média móvel

Com a base de dados devidamente ajustada, podemos calcular a média móvel dos últimos 5 jogos de cada time

In [39]:
#Cálculo da média móvel
df_mm = df_atualizado.groupby(['ano', 'clube'])[df_atualizado.iloc[:, 3:].columns].rolling(5).mean()

#Defasagem de variáveis
df_mm = df_mm[df_mm.columns.difference(['ano', 'clube'])].groupby(['ano', 'clube']).shift(1).reset_index()

#Renomeando coluna
df_mm = df_mm.rename(columns = {'level_2':'index'})

#Juntando as duas bases de dado
df_atualizado = df_atualizado.reset_index().iloc[:, :4].merge(df_mm)
df_atualizado

,level_0,index,partida_id,ano,clube,cartao_amarelo,cartao_vermelho,chutes,chutes_no_alvo,escanteios,faltas,impedimentos,passes,posse_de_bola,precisao_passes,rodata
0,0,0,6126,2018,Cruzeiro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,6126,2018,Gremio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,6127,2018,Vitoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,6127,2018,Flamengo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,6128,2018,Santos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,3795,3795,8023,2022,Atletico-MG,2.12,0.00,12.64,2.92,8.96,13.04,1.68,503.16,0.6016,0.8360,35.0
3796,3796,3796,8024,2022,Palmeiras,2.40,0.08,18.28,7.76,9.56,10.96,2.72,439.40,0.5624,0.8448,35.0
3797,3797,3797,8024,2022,Internacional,1.68,0.00,14.28,4.92,5.12,10.80,1.92,466.04,0.5368,0.8236,35.0
3798,3798,3798,8025,2022,Goias,3.60,0.00,11.32,4.20,4.24,16.32,1.76,306.88,0.4100,0.7328,34.8


## Aproveitamento

#### Ajeitar dataframe

Agora precisamos calcular o aproveitamento dos times. Para tal, precisamos dividir o número de pontos obtido pelo número de pontos possíveis de serem conquistados até aquele momento

In [150]:
brasileirao

,partida_id,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,resultado,ano,casa,pontos
0,6126,1,14/4/2018,16:00,Cruzeiro,Gremio,4-2-3-1,4-2-3-1,L. Venker de Menezes,Renato Gaúcho,Gremio,Mineirão,0,1,MG,RS,1,2018,0,3
1,6127,1,14/4/2018,19:02,Vitoria,Flamengo,4-2-3-1,4-1-4-1,V. Carmo Mancini,M. Nogueira Barbieri,-,Barradão,2,2,BA,RJ,2,2018,0,1
2,6128,1,14/4/2018,21:03,Santos,Ceara,4-3-3,4-1-4-1,J. Ribeiro Ventura,M. Oliveira Chamusca,Santos,Estádio Municipal Paulo Machado de Carvalho,2,0,SP,CE,0,2018,1,3
3,6129,1,15/4/2018,11:00,America-MG,Sport,4-1-4-1,4-3-2-1,E. Alves Moreira,N. Baptista Júnior,America-MG,Estádio Raimundo Sampaio,3,0,MG,PE,0,2018,1,3
4,6130,1,15/4/2018,16:01,Vasco,Atletico-MG,4-1-4-1,4-1-4-1,J. Mannarino,T. Mendes Larghi,Vasco,Estádio Vasco da Gama,2,1,RJ,MG,0,2018,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,8021,38,13/11/2022,16:03,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,A. Cardoso de Oliveira,A. Ferreira,Cuiaba,Arena Pantanal,2,1,MT,PR,0,2022,1,3
1896,8022,38,13/11/2022,16:03,Bragantino,Fluminense,4-2-3-1,4-2-3-1,M. Nogueira Barbieri,F. Diniz Silva,Fluminense,Nabizão,0,1,SP,RJ,1,2022,0,3
1897,8023,38,13/11/2022,16:03,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,F. J. Monteiro Almeida,A. Stival,Atletico-MG,Neo Química Arena,0,1,SP,MG,1,2022,0,3
1898,8024,38,13/11/2022,16:03,Internacional,Palmeiras,4-2-3-1,4-2-3-1,L. Venker de Menezes,A. Moreira Ferreira,Internacional,Estádio José Pinheiro Borda,3,0,RS,SP,0,2022,1,3


Terei de criar um novo dataframe para juntar com o *'df_atualizado'*. Devido ao número incompatível de colunas terei de realocar a coluna 'visitante' abaixo da 'mandante' e assim colar as duas

In [135]:
#Criar variável binária para identificar se um time está jogando em casa ou fora

brasileirao.loc[brasileirao['vencedor'] == brasileirao['mandante'], 'casa'] = 1
brasileirao.loc[brasileirao['vencedor'] != brasileirao['mandante'], 'casa'] = 0
brasileirao['casa'] = brasileirao['casa'].astype(int)

#Iniciar a coluna 'pontos'
brasileirao['pontos'] = 0

#Calcular os pontos conquistados pelos times
brasileirao.loc[(brasileirao['resultado'] == 0) & (brasileirao['casa'] == 1), 'pontos'] = 3
brasileirao.loc[(brasileirao['resultado'] == 1) & (brasileirao['casa'] == 0), 'pontos'] = 3
brasileirao.loc[(brasileirao['resultado'] == 2), 'pontos'] = 1

**A partir daqui criou-se o dataframe *df_final***

In [260]:
colunas = brasileirao[['mandante', 'visitante', 'vencedor', 'resultado', 'ano']]

mandantes = colunas['mandante']
visitantes = colunas['visitante']
resultado = colunas['resultado']
ano = colunas['ano']

times_alternados = pd.concat([mandantes, visitantes]).sort_index(kind='merge').reset_index(drop=True)

vencedores_binarios = pd.concat([(mandantes == colunas['vencedor']).astype(int),
                                (visitantes == colunas['vencedor']).astype(int)
                                ]).sort_index(kind='merge').reset_index(drop=True)
#Aqui acima concatenamos os times de forma alternada e identificamos o vencedor do jogo

#Criar variável binária para identificar se um time está jogando em casa ou fora
num_total = len(df_final)
coluna_alternada = (np.arange(num_total) % 2) ^ 1
coluna_alternada

#Criando o dataframe
df_final = pd.DataFrame({
    'times': times_alternados,
    'vencedor': vencedores_binarios,
    'casa': coluna_alternada,
    'resultado': resultado.repeat(2).reset_index(drop=True),
    'ano': ano.repeat(2).reset_index(drop=True)}) #resultado alternado

#Iniciar a coluna 'pontos'
df_final['pontos'] = 0

#Calcular os pontos conquistados pelos times
df_final.loc[(df_final['resultado'] == 0) & (df_final['casa'] == 1), 'pontos'] = 3
df_final.loc[(df_final['resultado'] == 1) & (df_final['casa'] == 0), 'pontos'] = 3
df_final.loc[(df_final['resultado'] == 2), 'pontos'] = 1

In [268]:
#Realiza a soma cumulativa dos pontos
df_aprov = df_final.groupby(['ano','times'])['pontos'].cumsum()
df_final['pontos_acum'] = df_aprov
df_final = df_final.drop(columns = 'pontos')

In [270]:
df_final

,times,vencedor,casa,resultado,ano,pontos_acum
0,Cruzeiro,0,1,1,2018,0
1,Gremio,1,0,1,2018,3
2,Vitoria,0,1,2,2018,1
3,Flamengo,0,0,2,2018,1
4,Santos,1,1,0,2018,3
...,...,...,...,...,...,...
3795,Atletico-MG,1,0,1,2022,58
3796,Internacional,1,1,0,2022,73
3797,Palmeiras,0,0,0,2022,81
3798,Goias,0,1,1,2022,46
